In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy
%matplotlib inline

In [5]:
data_path = '/nfshome/xueqin/deeplearning/plane_schedule/original_data.csv'

In [6]:
# pd.read_csv(data_path, delim_whitespace=True)

In [7]:
data_original = pd.read_csv(data_path, delim_whitespace=True)

In [8]:
coordinates = np.array([[565.0,575.0],[25.0,185.0],[345.0,750.0],[945.0,685.0],[845.0,655.0],
                        [880.0,660.0],[25.0,230.0],[525.0,1000.0],[580.0,1175.0],[650.0,1130.0],
                        [1605.0,620.0],[1220.0,580.0],[1465.0,200.0],[1530.0,  5.0],[845.0,680.0],
                        [725.0,370.0],[145.0,665.0],[415.0,635.0],[510.0,875.0],[560.0,365.0],
                        [300.0,465.0],[520.0,585.0],[480.0,415.0],[835.0,625.0],[975.0,580.0],
                        [1215.0,245.0],[1320.0,315.0],[1250.0,400.0],[660.0,180.0],[410.0,250.0],
                        [420.0,555.0],[575.0,665.0],[1150.0,1160.0],[700.0,580.0],[685.0,595.0],
                        [685.0,610.0],[770.0,610.0],[795.0,645.0],[720.0,635.0],[760.0,650.0],
                        [475.0,960.0],[95.0,260.0],[875.0,920.0],[700.0,500.0],[555.0,815.0],
                        [830.0,485.0],[1170.0, 65.0],[830.0,610.0],[605.0,625.0],[595.0,360.0],
                        [1340.0,725.0],[1740.0,245.0]])

In [9]:
coordinates.shape

(52, 2)

In [10]:
def getdistmat(coordinates):
    num = coordinates.shape[0]
    distmat = np.zeros((52, 52))
    for i in range(num):
        for j in range(i, num):
            distmat[i][j] = distmat[j][i] = np.linalg.norm(coordinates[i]-coordinates[j])
    return distmat

In [11]:
distmat = getdistmat(coordinates)

上面的式子计算了两个地点之间的距离, 并且写成了一个距离的矩阵, 距离公式是P=2的范数.

In [12]:
distmat.shape

(52, 52)

In [13]:
numant = 40 # 蚂蚁的个数

In [15]:
numcity = coordinates.shape[0] #城市的个数

In [16]:
alpha = 1  # 信息重要程度因子? 这是什么?

In [17]:
beta = 5 # 启发函数因子

In [18]:
rho = 0.1 # 信息素挥发程度

In [19]:
Q = 1 # 信息素常数

In [20]:
iter = 0 

In [21]:
itermax = 200

In [22]:
etatable = 1.0/(distmat+np.diag([1e10]*numcity))

etatable是启发函数矩阵, 就是每对城市的转移的概率.

计算的方法
对角矩阵, 对角线元素为1e10
对角矩阵加上每对城市距离, 再求倒数得到概率, 距离越大, 概率越小

In [23]:
pheromonetable = np.ones((numcity, numcity)) # 信息素矩阵, 一个全是1的两两城市矩阵

In [24]:
pathtable = np.zeros((numant, numcity)).astype(int) # 路径记录表 (每一只蚂蚁走过的城市顺序)

In [25]:
lengthaver = np.zeros(itermax) # 各代路径的平均长度. 200次迭代的蚂蚁们走过的平均长度

In [26]:
lengthbest = np.zeros(itermax) # 用来记录200次迭代的最佳长度, 也就是每次的最短长度.

In [27]:
pathbest = np.zeros((itermax, numcity)) # 每次迭代最短路径的城市顺序

In [28]:
np.random.permutation(range(0, numcity))[:numant]

array([46, 12, 37, 15, 34,  2, 20, 32, 25, 21, 19, 31, 22, 45, 26,  3, 40,
       51, 13, 23, 39, 16, 30, 10, 28, 17, 29,  8, 43,  4, 18, 49, 11, 24,
       14,  0, 48,  6, 33, 36])

In [29]:
print('机')

机


In [30]:
iter

0

In [31]:
%pdb

Automatic pdb calling has been turned ON


In [32]:
while iter < itermax:
    # 随机产生蚂蚁的起点城市
    if numant <= numcity:
        pathtable[:, 0] = np.random.permutation(range(0, numcity))[:numant] # 把起始城市按照蚂蚁数量随机分配
    else: # 对于蚂蚁比城市多的情况, 也是把所有城市随机分给每一个蚂蚁
        pathtable[:numcity, 0] = np.random.permutation(range(0, numcity))[:] # 先分配前面的蚂蚁, 保证每个城市都可能作为起点
        pathtable[numcity:, 0] = np.random.permutation(range(0, numcity))[:numant - numcity] # 再分配多出的蚂蚁, 随机分配
    length = np.zeros(numant) # 蚂蚁的路径距离
    for i in range(numant): # 对于每一只蚂蚁
        visiting = pathtable[i, 0] # 起始城市的位置, 就是之前随机分配的城市.
        # visited = set() # 使用set记录所有去过的城市, 因为set是集合, 集合中的元素不可以重复.
        # visited.add(visiting) # 每去一个城市, 把当前城市添加到集合里面.
        unvisited = set(range(numcity)) # 创建为访问的城市集合
        unvisited.remove(visiting) # 删除访问过的城市, 留下的就是没有访问的
        
        for j in range(1, numcity): # 访问完所有的城市需要的步数
            # 下面是轮盘法的代码, 使用轮盘法选择下一个城市
            # 轮盘法可能是一个选取城市的一个公式.
            listunvistited = list(unvisited)
            probtrans = np.zeros(len(listunvistited))
            
            for k in range(len(listunvistited)):
                # 这个下面就是那个很复杂的方程.t时刻,蚂蚁k从城市i到城市j的概率的分子
                probtrans[k] = np.power(pheromonetable[visiting][listunvistited[k]], alpha)*np.power(etatable[visiting][listunvistited[k]], beta)
                # @TODO: 这里他写的是alpha, 按照公式应该是beta.
            # 计算概率
            cumsumprobtrans = (probtrans/sum(probtrans)).cumsum()
            # 概率随机减去一点
            cumsumprobtrans -= np.random.rand() # 减去的这一点可以让某些概率为负值.
            k = listunvistited[np.where(cumsumprobtrans>0)[0][0]] # 下一个访问的城市. 城市为概率大于0的第一个城市.
            # @TODO: 这个地方可能会有bug, 评论里面说的
            pathtable[i,j] = k # 记录第i只蚂蚁在第j步访问的城市.
            unvisited.remove(k) # 移除访问过的城市
            # visited.add(k)
            
            length[i] += distmat[visiting][k] # 把蚂蚁之前城市到目前城市的距离算到总路程.
            visiting = k # 目前城市定位到k
            
        length[i] += distmat[visiting][pathtable[i, 0]] # 从目前的城市回归起点城市的距离.
        # print("第{}蚂蚁总共走了{}米".format(i, length[i]))
    
    # 包含所有蚂蚁的一个迭代结束后, 统计这次迭代的统计参数.
    lengthaver[iter] = length.mean() # 所有的蚂蚁在这一个循环中走过的平均距离
    if iter == 0:
        lengthbest[iter] = length.min()
        pathbest[iter] = pathtable[length.argmin()].copy() # 走过路径最短的那个蚂蚁的路线作为最佳路线
    else:
        if length.min() > lengthbest[iter-1]: # 如果当前的最短路径大于上一次迭代的最短路径
            lengthbest[iter] = lengthbest[iter-1] # 最短的路径依然是上次迭代的结果
            pathbest[iter] = pathbest[iter-1].copy() # 最佳路径依然是之前的路径
        else: # 如果当前路径最佳,更新最佳路径和最佳距离
            lengthbest[iter] = length.min()
            pathbest[iter] = pathtable[length.argmin()].copy()
            print("最短路径{}米".format(lengthbest[iter]))
            print("经过城市{}".format(pathbest[iter]))
    # 更新信息素
    change_pheromone_table = np.zeros((numcity, numcity))
    for i in range(numant):
        for j in range(numcity-1):
            change_pheromone_table[pathtable[i,j]][pathtable[i, j+1]] += Q/distmat[pathtable[i,j]][pathtable[i, j+1]]
        change_pheromone_table[pathtable[i, j+1]][pathtable[i, 0]] += Q/distmat[pathtable[i, j+1]][pathtable[i, 0]] # 从最后一个城市回到起点
    pheromonetable = (1-rho)*pheromonetable + change_pheromone_table
    
    iter += 1
    
    if (iter-1)%20 == 0:
        print("当前迭代次数{}".format(iter-1))        

当前迭代次数0
最短路径8986.217016046952米
经过城市[19. 49. 15. 43. 35. 34. 33. 39. 37. 23. 47.  4. 14.  5.  3. 24. 36. 38.
 31. 48.  0. 21. 30. 17.  2. 18. 44. 40.  7.  8.  9. 42. 32. 50. 11. 27.
 26. 25. 46. 12. 13. 51. 10. 45. 28. 22. 29. 41.  6.  1. 20. 16.]
最短路径8960.151224975372米
经过城市[30. 17. 21.  0. 48. 31. 38. 39. 37. 36. 23. 47.  4. 14.  5.  3. 24. 11.
 27. 26. 25. 46. 13. 12. 51. 10. 50. 32. 42.  8.  9.  7. 40. 18. 44.  2.
 20. 16. 45. 43. 33. 34. 35. 15. 49. 19. 22. 29. 28.  1.  6. 41.]
最短路径8793.259610125902米
经过城市[47. 23.  4. 14.  5.  3. 24. 45. 43. 33. 34. 35. 38. 39. 37. 36.  0. 21.
 48. 31. 44. 18. 40.  7.  9.  8. 42. 32. 50. 11. 27. 26. 25. 46. 12. 13.
 51. 10. 15. 19. 49. 22. 30. 17.  2. 16. 20. 41.  6.  1. 29. 28.]
最短路径8593.334335651985米
经过城市[22. 19. 49. 15. 43. 34. 35. 33. 38. 39. 37. 36. 23. 47.  4. 14.  5.  3.
 24. 11. 50. 25. 26. 27. 46. 13. 12. 51. 10. 32. 42.  9.  8.  7. 40. 18.
 44. 31. 48.  0. 21. 30. 17.  2. 16. 20. 41.  6.  1. 29. 28. 45.]
最短路径8560.220759272916米
经过城市[39. 37. 